There are n piles of stones arranged in a row. The ith pile has stones[i] stones.

A move consists of merging exactly k consecutive piles into one pile, and the cost of this move is equal to the total number of stones in these k piles.

Return the minimum cost to merge all piles of stones into one pile. If it is impossible, return -1.

 

Example 1:

Input: stones = [3,2,4,1], k = 2
Output: 20
Explanation: We start with [3, 2, 4, 1].
We merge [3, 2] for a cost of 5, and we are left with [5, 4, 1].
We merge [4, 1] for a cost of 5, and we are left with [5, 5].
We merge [5, 5] for a cost of 10, and we are left with [10].
The total cost was 20, and this is the minimum possible.

Example 2:

Input: stones = [3,2,4,1], k = 3
Output: -1
Explanation: After any merge operation, there are 2 piles left, and we can't merge anymore.  So the task is impossible.

Example 3:

Input: stones = [3,5,1,2,6], k = 3
Output: 25
Explanation: We start with [3, 5, 1, 2, 6].
We merge [5, 1, 2] for a cost of 8, and we are left with [3, 8, 6].
We merge [3, 8, 6] for a cost of 17, and we are left with [17].
The total cost was 25, and this is the minimum possible.

 

Constraints:

    n == stones.length
    1 <= n <= 30
    1 <= stones[i] <= 100
    2 <= k <= 30



In [46]:
from typing import List

class Solution:
    def mergeStones(self, stones: List[int], k: int) -> int:
        if (len(stones) - 1) % (k - 1) != 0:
            return -1
        
        sums = []
        for i in range(len(stones) - k + 1):
            sums.append(sum(stones[i:i + k]))
        
        cost = 0
        while len(stones) > 1:
            idx = sums.index(min(sums))
            cost += sums[idx]
            stones[idx] = sum(stones[idx:idx + k])
            for i in range(1, k)[::-1]:
                del stones[idx + i]
                if idx + i < len(sums):
                    del sums[idx + i]
                sums = []
                for j in range(len(stones) - k + 1):
                    sums.append(sum(stones[j:j + k]))

        return cost

In [1]:
from typing import List
from functools import cache

class Solution:
    def mergeStones(self, stones: List[int], k: int) -> int:
        if (len(stones) - 1) % (k - 1) != 0:
            return -1
        
        @cache
        def merge_stones(stones, cost, k):
            if len(stones) == 1:
                return cost
            
            stones = list(stones)
            costs = []
            for i in range(len(stones) - k + 1):
                stones_curr = stones.copy()
                cost_curr = sum(stones[i:i + k])
                for j in range(i + 1, i + k)[::-1]:
                    del stones_curr[j]
                stones_curr[i] = cost_curr
                costs.append(merge_stones(tuple(stones_curr), cost + cost_curr, k))

            return min(costs)
        
        return merge_stones(tuple(stones), 0, k)
        

In [9]:
from itertools import accumulate
from functools import lru_cache

class Solution:
    def mergeStones(self, stones: List[int], k: int) -> int:
        #every merge reduces number of stones by k - 1
        if (len(stones) - 1) % (k - 1) != 0: 
            return -1
        acc = list(accumulate(stones))
        sumrange = lambda i, j: acc[j] - (acc[i - 1] if i > 0 else 0)
        
        # cost to merge stones[i:j + 1] into n piles
        @lru_cache(None)
        def dp(i, j, n):
            lenrange = j - i + 1
            if lenrange == n: 
                return 0
            if lenrange < n or (lenrange - n) % (k - 1) != 0: 
                return 1000000
            if n == 1: 
                return dp(i, j, k) + sumrange(i, j)
            
            return min(dp(i, t, 1) + dp(t + 1, j, n - 1) for t in range(i, j))
        return dp(0, len(stones) - 1, 1)

In [11]:
from math import inf
from functools import cache

class Solution:
    def mergeStones(self, stones: List[int], k: int) -> int:
        if (len(stones) - 1) % (k - 1): 
            return -1 # impossible
        
        prefix = [0]
        for x in stones: 
            prefix.append(prefix[-1] + x)
        
        @cache
        def fn(lo, hi): 
            """Return min cost of merging stones[lo:hi]."""
            if hi - lo < k: 
                return 0 # not enough stones
            ans = inf 
            for mid in range(lo + 1, hi, k - 1): 
                ans = min(ans, fn(lo, mid) + fn(mid, hi))
            if (hi - lo - 1) % (k - 1) == 0: 
                ans += prefix[hi] - prefix[lo]
            return ans 
        
        return fn(0, len(stones))

In [12]:
s = Solution()
s.mergeStones([3, 5, 1, 2, 6], 3)

25

In [13]:
s.mergeStones([5, 1, 2], 3)

8

In [14]:
s.mergeStones([6,4,4,6], 2)

40

In [15]:
s.mergeStones([69,39,79,78,16,6,36,97,79,27,14,31,4], 2)

1957

In [6]:
s.mergeStones([29,59,31,7,51,99,47,40,24,20,98,41,42,81,92,55], 2)

3152